In [14]:
import sys
print(sys.executable)

c:\Users\claud\GIT\Proyecto_Hoteles_ETL\venv\Scripts\python.exe


In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Configuración de Selenium
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--headless")  # Ejecutar sin ventana para optimizar velocidad

# Iniciar el navegador
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# URL de la página de hoteles en Madrid
url = "https://all.accor.com/ssr/app/ibis/hotels/madrid-spain/open/index.es.shtml?dateIn=2025-03-01&nights=1&destination=madrid-spain"
driver.get(url)

# Esperar a que los hoteles carguen
try:
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CLASS_NAME, "hotelblock__content"))
    )
    print("✅ Lista de hoteles encontrada.")
except:
    print("❌ No se encontraron hoteles. Puede que la página tarde en cargar.")
    driver.quit()
    exit()

# Extraer todos los hoteles
hoteles = driver.find_elements(By.CLASS_NAME, "hotelblock__content")

print(f"🔍 Se encontraron {len(hoteles)} hoteles.")

datos_hoteles = []

# **Iteramos por cada hotel en la lista**
for index in range(len(hoteles)):  
    try:
        # **Reubicar el elemento del hotel en cada iteración para evitar el error "stale element reference"**
        hoteles = driver.find_elements(By.CLASS_NAME, "hotelblock__content")  # Recargar lista
        hotel = hoteles[index]  

        # Hacer scroll hasta el hotel para asegurarse de que el precio aparece
        driver.execute_script("arguments[0].scrollIntoView();", hotel)
        time.sleep(1)  # Pequeña pausa para permitir la carga del contenido

        # Extraer el nombre del hotel con `WebDriverWait`
        try:
            nombre = WebDriverWait(hotel, 3).until(
                EC.presence_of_element_located((By.CLASS_NAME, "title-block"))
            ).text.strip()
        except:
            nombre = "No disponible"

        # Extraer la valoración de usuarios
        try:
            valoracion = WebDriverWait(hotel, 3).until(
                EC.presence_of_element_located((By.CLASS_NAME, "hotelblock__content-ratings"))
            ).text.strip()
        except:
            valoracion = "No disponible"

        # Intentar extraer el primer precio disponible
        precio = "No disponible"

        try:
            precio_container = WebDriverWait(hotel, 3).until(
                EC.presence_of_element_located((By.CLASS_NAME, "rate-details__price1"))
            )
            precio = precio_container.find_element(By.CLASS_NAME, "booking-price__number").text.strip()
        except:
            pass  # Si no encuentra precio en `rate-details__price1`, sigue buscando en `rate-details__price2`

        if precio == "No disponible":
            try:
                precio_container = WebDriverWait(hotel, 3).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "rate-details__price2"))
                )
                precio = precio_container.find_element(By.CLASS_NAME, "booking-price__number").text.strip()
            except:
                pass  # Si tampoco lo encuentra en `rate-details__price2`, sigue como "No disponible"

        # Guardar datos en lista
        datos_hoteles.append({
            "nombre_hotel": nombre,
            "estrellas": valoracion,
            "precio_noche": precio
        })

        print(f"{index+1}. 🏨 {nombre} - ⭐ {valoracion} - 💰 {precio}")

    except Exception as e:
        print(f"❌ Error al extraer datos del hotel {index+1}: {e}")

# Cerrar el navegador
driver.quit()

# Convertir en DataFrame
df_hoteles = pd.DataFrame(datos_hoteles)
df_hoteles

✅ Lista de hoteles encontrada.
🔍 Se encontraron 10 hoteles.
1. 🏨 ibis Styles Madrid Prado
3 Estrellas
Hoteles
0.4 km desde tu búsqueda - ⭐ Puntuación de viajeros de TripAdvisor:
4.7/
En
5
ver reseñas - 💰 161
2. 🏨 No disponible - ⭐ No disponible - 💰 No disponible
3. 🏨 ibis Madrid Centro las Ventas
2 Estrellas
Hoteles
3.7 km desde tu búsqueda - ⭐ Puntuación de viajeros de TripAdvisor:
4.5/
En
5
ver reseñas - 💰 172
4. 🏨 Novotel Madrid City Las Ventas
4 Estrellas
Hoteles
4.7 km desde tu búsqueda - ⭐ Puntuación de viajeros de TripAdvisor:
4.6/
En
5
ver reseñas - 💰 173
5. 🏨 ibis budget Madrid Vallecas
1 Estrellas
Hoteles
7.2 km desde tu búsqueda - ⭐ Puntuación de viajeros de TripAdvisor:
4.3/
En
5
ver reseñas - 💰 107
6. 🏨 ibis Madrid Aeropuerto Barajas
2 Estrellas
Hoteles
12.0 km desde tu búsqueda - ⭐ Puntuación de viajeros de TripAdvisor:
4.4/
En
5
ver reseñas - 💰 116
7. 🏨 ibis Madrid Alcorcon Tresaguas
2 Estrellas
Hoteles
12.1 km desde tu búsqueda - ⭐ Puntuación de viajeros de TripAdvisor:

,nombre_hotel,estrellas,precio_noche
0,ibis Styles Madrid Prado\n3 Estrellas\nHoteles...,Puntuación de viajeros de TripAdvisor:\n4.7/\n...,161
1,No disponible,No disponible,No disponible
2,ibis Madrid Centro las Ventas\n2 Estrellas\nHo...,Puntuación de viajeros de TripAdvisor:\n4.5/\n...,172
3,Novotel Madrid City Las Ventas\n4 Estrellas\nH...,Puntuación de viajeros de TripAdvisor:\n4.6/\n...,173
4,ibis budget Madrid Vallecas\n1 Estrellas\nHote...,Puntuación de viajeros de TripAdvisor:\n4.3/\n...,107
5,ibis Madrid Aeropuerto Barajas\n2 Estrellas\nH...,Puntuación de viajeros de TripAdvisor:\n4.4/\n...,116
6,ibis Madrid Alcorcon Tresaguas\n2 Estrellas\nH...,Puntuación de viajeros de TripAdvisor:\n4.4/\n...,90
7,ibis budget Madrid Aeropuerto\n1 Estrellas\nHo...,Puntuación de viajeros de TripAdvisor:\n4/\nEn...,88
8,ibis Madrid Alcobendas\n2 Estrellas\nHoteles\n...,Puntuación de viajeros de TripAdvisor:\n4.4/\n...,85
9,ibis budget Madrid Alcorcon Móstoles\n2 Estrel...,Puntuación de viajeros de TripAdvisor:\n4.5/\n...,77


In [12]:
# Limpiar DataFrame
 
import pandas as pd
import re

def limpiar_dataframe(df):
    """
    Limpia el DataFrame:
    - Extrae solo el nombre del hotel de la columna "nombre_hotel".
    - Extrae solo la puntuación de la columna "estrellas" y la convierte a un formato decimal.
    """
    
    # Limpiar "nombre_hotel"
    df["nombre_hotel"] = df["nombre_hotel"].str.split("\n").str[0]  # Separa por salto de línea y toma solo el primer valor
    
    # Limpiar "estrellas" y asegurarnos de que tenga formato decimal
    def limpiar_valoracion(valor):
        match = re.search(r"\d+(\.\d+)?", valor)  # Buscar número con o sin decimal
        if match:
            num = match.group(0)  # Extraer el número
            return f"{float(num):.1f}"  # Convertir a float con un decimal
        return "No disponible"  # Si no hay número, mantener como "No disponible"

    df["estrellas"] = df["estrellas"].apply(limpiar_valoracion)
    
    return df

# Aplicar la función al DataFrame
df_hoteles = limpiar_dataframe(df_hoteles)
 
# Guardar el DataFrame limpio en CSV
df_hoteles.to_csv("../data/data_raw/hoteles_competencia.csv", index=False, encoding="utf-8")

In [13]:
df_hoteles

,nombre_hotel,estrellas,precio_noche
0,ibis Styles Madrid Prado,4.7,161
1,Novotel Madrid Center,4.6,255
2,ibis Madrid Centro las Ventas,4.5,172
3,Novotel Madrid City Las Ventas,4.6,173
4,ibis budget Madrid Vallecas,4.3,107
5,ibis Madrid Aeropuerto Barajas,4.4,116
6,ibis Madrid Alcorcon Tresaguas,4.4,90
7,ibis budget Madrid Aeropuerto,4.0,88
8,ibis Madrid Alcobendas,4.4,85
9,ibis budget Madrid Alcorcon Móstoles,4.5,77
